In [27]:
import os
import json
import openai
import tiktoken
from langchain.chat_models import ChatOpenAI
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.document_loaders.word_document import Docx2txtLoader 


In [8]:
# test your API key and configuration
with open('../../.secrets/yyds.json') as f:
    secrets = json.load(f)
os.environ['OPENAI_API_KEY'] = secrets['api_key']
os.environ['OPENAI_API_BASE'] = secrets['api_base']

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
llm([HumanMessage(content="Tell me a joke.")])

AIMessage(content="Sure, here's a classic one for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!", additional_kwargs={}, example=False)

In [25]:
loader = Docx2txtLoader('./Sample-Redemption-EN.docx')
docs_all = loader.load()
enc = tiktoken.encoding_for_model('gpt-3.5-turbo')
tokens = enc.encode(docs_all[0].page_content)
print(len(tokens))

1899


In [29]:
summary_prompt = PromptTemplate.from_template("""You're a helpful AI assistant to help lawers to write concise summary of given document.
----the document is about {words} words long----
{document}
----the summary to be generated----
""")
if len(tokens) < 3*1024: # can be processed in a 4K window
    chain = LLMChain(llm=llm, prompt=summary_prompt)
    result = chain({'words': int(len(tokens)*0.75), 'document': docs_all[0].page_content})
    summary_text = result['text']

{'words': 1424, 'document': 'REDEMPTION\n\n\tRedemption by the Company.  Notwithstanding anything to the contrary herein, at any time after the earlier of (A) following the fifth (5th) year anniversary of the Series Y Original Issue Date, (B) any equity securities of any Group Company become redeemable, (C) any material adverse change in the regulatory environment which will result the Control Documents becomes invalid, or (D) any material breach by any Group Company or any Founder Party of any terms under the Transaction Documents, including without limitation, any breach of non-compete, non-solicitation, full time commitment obligations by the Founder (the “Redemption Start Date”), subject to the applicable laws of the Cayman Islands and, if so requested by the Series Y Majority, the Company shall redeem all or any part of the outstanding Preferred Shares held by such holder out of funds legally available therefor (the “Redemption”).  \n\nA.\tThe price at which each Series Y Preferre

In [34]:
with open('./sample_summary.txt', 'w') as f:
    f.write(summary_text)